In [ ]:
import numpy as np
import unyt
import swiftsimio as sw
from velociraptor import load
import swiftgalaxy as sg
import functions as fn
from swiftgalaxy import SWIFTGalaxy, MaskCollection
import h5py
dir="/mnt/su3-pro/flamingo/L0200N0360/"

data_h=load(dir+"VR/halos_0008.properties.0")
radius=data_h.radii.rvir

is_mainhalo=data_h.centrals
halo_id=data_h.ids.id-1
mass=data_h.masses.mass_tot
xc=data_h.positions.xcminpot
yc=data_h.positions.ycminpot
zc=data_h.positions.zcminpot
data_h=[]
mainhalo_id=halo_id[is_mainhalo*(mass>10000)]

mainhalo_id=mainhalo_id.astype(int)




In [20]:
#S>0.2:24.,  25.,  28.,  29.,  32., 101., 102.
#S<0:8., 56., 75., 95., 99
#Offset<0.01:11., 16., 34., 50
#Offset>0.06:72.,  73.,  87., 105
#CoM different>0.15: 1.,   2.,  10.,  21.,  57.,  73.,  85., 105

for id in mainhalo_id:
  

  centre=np.array([xc[id],yc[id],zc[id]])*unyt.Mpc
  r=radius[id]
 
  sgi=sg.SWIFTGalaxy("/../../mnt/su3-pro/flamingo/L0200N0720/snapshots/flamingo_0008/flamingo_0008.hdf5",
                   sg.Standalone(centre=centre,velocity_centre=np.array([0,0,0])*
                                 unyt.km/unyt.s,spatial_offsets=[[-r,r],[-r,r],[-r,r]],extra_mask=None))
#  print(len(sgi.stars.spherical_coordinates.r))
  mask=sg.MaskCollection(dark_matter=(sgi.dark_matter.spherical_coordinates.r<r),
                       gas=(sgi.gas.spherical_coordinates.r<r)*(sgi.gas.temperatures>00000),
                       
                       stars=sgi.stars.spherical_coordinates.r<r)
  #
  sgi.mask_particles(mask)
#  print(len(sgi.stars.spherical_coordinates.r))
  xdm=np.array(sgi.dark_matter.cartesian_coordinates.x)
  ydm=np.array(sgi.dark_matter.cartesian_coordinates.y)
  zdm=np.array(sgi.dark_matter.cartesian_coordinates.z)
  PartType2=np.array([xdm,ydm,zdm]).T
  
  p=np.array(sgi.dark_matter.potentials)
  mbp=PartType2[p==np.min(p)][0]
  print(mbp)
  if fn.radial_distance(mbp[0],mbp[1],mbp[2])>0.01*float(r):
    centre+=mbp*unyt.Mpc
    sgi=sg.SWIFTGalaxy("/../../mnt/su3-pro/flamingo/L0200N0720/snapshots/flamingo_0008/flamingo_0008.hdf5",
                   sg.Standalone(centre=centre,velocity_centre=np.array([0,0,0])*
                                 unyt.km/unyt.s,spatial_offsets=[[-r,r],[-r,r],[-r,r]],extra_mask=None))
    mask=sg.MaskCollection(dark_matter=(sgi.dark_matter.spherical_coordinates.r<r),
                       gas=(sgi.gas.spherical_coordinates.r<r)*(sgi.gas.temperatures>00000),
                       
                       stars=sgi.stars.spherical_coordinates.r<r)
    sgi.mask_particles(mask)
    xdm=np.array(sgi.dark_matter.cartesian_coordinates.x)
    ydm=np.array(sgi.dark_matter.cartesian_coordinates.y)
    zdm=np.array(sgi.dark_matter.cartesian_coordinates.z)  
    PartType2=np.array([xdm,ydm,zdm]).T
  xg=np.array(sgi.gas.cartesian_coordinates.x)
  yg=np.array(sgi.gas.cartesian_coordinates.y)
  zg=np.array(sgi.gas.cartesian_coordinates.z)
  PartType1=np.array([xg,yg,zg]).T
  xstar=np.array(sgi.stars.cartesian_coordinates.x)
  ystar=np.array(sgi.stars.cartesian_coordinates.y)
  zstar=np.array(sgi.stars.cartesian_coordinates.z)
  PartType3=np.array([xstar,ystar,zstar]).T
  
  dm_potentials=np.array(sgi.dark_matter.potentials)
  gas_potentials=np.array(sgi.gas.potentials)
  gas_temperatures=np.array(sgi.gas.temperatures)
  gas_densities=np.array(sgi.gas.densities)
  star_lum=np.array(sgi.stars.luminosities.GAMA_z)
  star_potentials=np.array(sgi.stars.potentials)
  a_last_agn=np.array(sgi.gas.last_agnfeedback_scale_factors)
  a_max_t=np.array(sgi.gas.maximal_temperature_scale_factors)
  xray_lum_high=np.array(sgi.gas.xray_luminosities.erosita_high)
  xray_lum_low=np.array(sgi.gas.xray_luminosities.erosita_low)
#  r_dm=sgi.dark_matter.spherical_coordinates.r/r


  f = h5py.File('/home/jyang/data/Flamingo/L0200N0360/halo_particles_recenter/'+str(int(id))+'.hdf5', 'w')
  s=f.create_group("PartType2")#star
  s.create_dataset("Coordinates", data=PartType3)
  s.create_dataset("lum_gamaz", data=star_lum)
  s.create_dataset("potentials", data=star_potentials)
  dm = f.create_group("PartType1")#dm
  dm.create_dataset("Coordinates", data=PartType2)
  dm.create_dataset("potentials", data=dm_potentials)
  g= f.create_group("PartType0")#gas
#  g=f["PartType0"]
  g.create_dataset("Coordinates", data=PartType1)
  g.create_dataset("potentials", data=gas_potentials)
  g.create_dataset("temperatures", data=gas_temperatures)
  g.create_dataset("densities", data=gas_densities)
  g.create_dataset("xray_lum_erosita_high", data=xray_lum_high)
  g.create_dataset("xray_lum_erosita_low", data=xray_lum_low)
  g.create_dataset("a_agn", data=a_last_agn)
#  g.create_dataset("star_id", data=star_id)

  g.create_dataset("a_max_t", data=a_max_t)
  f.create_dataset("Header", data=[0,0,0,0,0,0,0,0])
  f.attrs['comment']="xray_lum.units=10.697030298874*Mpc**2*Msun/Gyr**3."
  f.attrs['comment']+="densities.units=10000000000.0*Msun/Mpc**3."
  f.attrs['comment']+="temperatures.units=K"
#  f.attrs['comment']+="gas id, the scale factor of last AGN feedback, the scale factor of maximal temperature"
  f.close()


/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor (a at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftgalaxy/reader.py:2087: RuntimeWarning: Translation assumed to be in comoving (not physical) coordinates.
  warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:665: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor ((a**2.0)**0.5 at a=1.0) for all arguments.
  warnings.warn(


[ 0.05513371 -0.04340635  0.03424119]


/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor (a at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftgalaxy/reader.py:2087: RuntimeWarning: Translation assumed to be in comoving (not physical) coordinates.
  warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/unyt/array.py:1972: RuntimeWarning: invalid value encountered in divide
  out_arr = func(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:665: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor ((a**2.0)**0.5 at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without 

[ 0.00610236 -0.02222382  0.00638363]


/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor (a at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftgalaxy/reader.py:2087: RuntimeWarning: Translation assumed to be in comoving (not physical) coordinates.
  warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:665: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor ((a**2.0)**0.5 at a=1.0) for all arguments.
  warnings.warn(


...

/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor (a at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftgalaxy/reader.py:2087: RuntimeWarning: Translation assumed to be in comoving (not physical) coordinates.
  warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:665: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor ((a**2.0)**0.5 at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor (a at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.1

[-0.05429273 -0.01182243  0.01570267]


/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor (a at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftgalaxy/reader.py:2087: RuntimeWarning: Translation assumed to be in comoving (not physical) coordinates.
  warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/unyt/array.py:1972: RuntimeWarning: invalid value encountered in divide
  out_arr = func(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:665: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor ((a**2.0)**0.5 at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without 

[ 0.01570121 -0.01275262 -0.02960319]


/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor (a at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftgalaxy/reader.py:2087: RuntimeWarning: Translation assumed to be in comoving (not physical) coordinates.
  warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/unyt/array.py:1972: RuntimeWarning: invalid value encountered in divide
  out_arr = func(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:665: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor ((a**2.0)**0.5 at a=1.0) for all arguments.
  warnings.warn(


In [ ]:
f = h5py.File('/home/jyang/data/Flamingo/L0200N0360/halo_particles/66test.hdf5', 'w')
dm = f.create_group("PartType2")
dm.create_dataset("Coordinates", data=PartType1)
g= f.create_group("PartType1")
g.create_dataset("Coordinates", data=PartType2)
g.create_dataset("xray", data=PartType2)
f.create_dataset("Header", data=[0,0,0,0,0,0,0,0])
f.close()